In [1]:
# 데이터 처리 라이브러리
import os
import os.path as pth
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split

# Tensorflow 관련 라이브러리
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, BatchNormalization, Flatten, Activation
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam


# GPU 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # 특정 GPU에 1GB 메모리만 할당하도록 제한
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        
        tf.config.experimental.set_memory_growth(gpus[0], True)
        
#         tf.config.experimental.set_virtual_device_configuration(
#             gpus[0],
#             [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=15000)])
    except RuntimeError as e:
    # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
        print(e)

In [ ]:
train_tfrecord_path = pth.join('./data/emotion_img_tf2', 'tf_record_train.tfrecords')
valid_tfrecord_path = pth.join('./data/emotion_img_tf2', 'tf_record_train.tfrecords')

# BUFFER_SIZE, BATCH_SIZE
BUFFER_SIZE = 100
BATCH_SIZE = 20
NUM_CLASS = 8



image_feature_description = {
    'image_raw': tf.io.FixedLenFeature([], tf.string),
    'img_id': tf.io.FixedLenFeature([], tf.string),
}



def _parse_image_function(example_proto):
    return tf.io.parse_single_example(example_proto, image_feature_description)

def map_func(target_record):
    img = target_record['image_raw']
    label = int(target_record['img_id'])
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.dtypes.cast(img, tf.float32)
    return img, label

def prep_func(image, label):
    print(type(label))

    result_image = image / 255
    result_image = tf.image.resize(result_image, (270,480))

    onehot_label = tf.one_hot(label, depth=NUM_CLASS)
    return result_image, onehot_label

    

dataset = tf.data.TFRecordDataset(train_tfrecord_path, compression_type='GZIP')
dataset = dataset.map(_parse_image_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.map(map_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.map(prep_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

valid_dataset = tf.data.TFRecordDataset(valid_tfrecord_path, compression_type='GZIP')
valid_dataset = valid_dataset.map(_parse_image_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.map(map_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.shuffle(BUFFER_SIZE)
valid_dataset = valid_dataset.batch(BATCH_SIZE)
valid_dataset = valid_dataset.map(prep_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [ ]:
earlystop = EarlyStopping(patience = 5)

learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',
                                            patience = 2,
                                            factor = 0.5,
                                            min_lr = 1e-7,
                                            verbose = 1)

model_check = ModelCheckpoint(filepath = '/model/facial_mobile_270_480_adam.h5',
                              monitor = 'val_loss',
                              save_best_only = True)

callbacks = [earlystop, laerning_rate_reduction, model_check]

In [ ]:
from tensorflow.keras.applications import MobileNetV2
mobilenet = MobileNetV2(weights = 'imagenet', include_top = False, input_shape = (270.480,3))


for layer in mobilenet.layers:
    layer.trainable = True
    
for (i, layer) in enumerate(mobilenet.layers):
    print(str(i), layer.__class__.__name__, layer.trainable)
    
    
model = Sequential()
model.add(mobilenet)
model.add(GlobalAveragePooling2D())
model.add(Dense(8))
model.add(BatchNormalization())
model.add(PReLU())
model.add(Softmax(dtype='float32', name='sortmax'))
model.summary()